<a href="https://colab.research.google.com/github/Taedriel/ZSL-v2/blob/visual2textual/visual_vectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing libraries :

In [3]:
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import AveragePooling2D, MaxPool2D, Dropout, Flatten, Dense, Input, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img 
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix

## Importing dataset :

In [6]:
import tensorflow_datasets as tfds

ds_name = 'oxford_flowers102'
splits = ['test', 'train']
(ds_train, ds_test), info = tfds.load(ds_name, 
                                      split = splits, 
                                      shuffle_files=True,
                                      as_supervised=True,
                                      with_info=True)


### Dataset info : 

In [7]:
print(f"Number of Classes : {info.features['label'].num_classes}")
print(f"Training data: {tf.data.experimental.cardinality(ds_train)}")
print(f"Test data: {tf.data.experimental.cardinality(ds_test)}\n")
print('Flower types full list:')
print(info.features['label'].names)

Number of Classes : 102
Training data: 6149
Test data: 1020

Flower types full list:
['pink primrose', 'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea', 'english marigold', 'tiger lily', 'moon orchid', 'bird of paradise', 'monkshood', 'globe thistle', 'snapdragon', "colt's foot", 'king protea', 'spear thistle', 'yellow iris', 'globe-flower', 'purple coneflower', 'peruvian lily', 'balloon flower', 'giant white arum lily', 'fire lily', 'pincushion flower', 'fritillary', 'red ginger', 'grape hyacinth', 'corn poppy', 'prince of wales feathers', 'stemless gentian', 'artichoke', 'sweet william', 'carnation', 'garden phlox', 'love in the mist', 'mexican aster', 'alpine sea holly', 'ruby-lipped cattleya', 'cape flower', 'great masterwort', 'siam tulip', 'lenten rose', 'barbeton daisy', 'daffodil', 'sword lily', 'poinsettia', 'bolero deep blue', 'wallflower', 'marigold', 'buttercup', 'oxeye daisy', 'common dandelion', 'petunia', 'wild pansy', 'primula', 'sunflower', 'pelargonium', '

In [8]:
num_classes = info.features['label'].num_classes

In [9]:
def normalize_img(img, label):
  img = tf.cast(img, tf.float32) / 255.
  return (img, label)

In [10]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 64

def preprocess_data(data, info, train=True, batch_size=BATCH_SIZE):
    data = data.map(normalize_img, num_parallel_calls=AUTOTUNE)
    if train:
        data = data.cache()
        data = data.shuffle(tf.data.experimental.cardinality(data))
    data = data.batch(batch_size)
    data = data.prefetch(AUTOTUNE)
    return data

In [11]:
ds_train = preprocess_data(ds_train, info)
ds_test = preprocess_data(ds_test, info, train=False, batch_size=128)

# The VGG Model : VGG16

In [12]:
base_model = VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

In [13]:
head_model = base_model.output
head_model = AveragePooling2D((7, 7))(head_model)
head_model = Flatten(name="flatten")(head_model)
head_model = Dense(units=num_classes, activation="softmax")(head_model)

## Making the VGG16 layers non trainable (Transfer learning):

In [14]:
model = Model(inputs=base_model.input, outputs=head_model)
base_model.trainable = False
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [15]:
now = datetime.datetime.now

def train_model(model, optimizer, ds_train, ds_test, modelName, epochs=10):
    print("Model summary :\n")
    model.summary()
    print("Compiling model...")
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  optimizer=optimizer,
                  metrics=['accuracy'])

    t = now()
    
    model.fit(ds_train, epochs=epochs,  
              validation_data=ds_test)
    print('Training time: %s' % (now() - t))

    print("[INFO] evaluating network...")
    score = model.evaluate(ds_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])
    
    #Saving the model :
    print("Saving the model...")
    model.save(modelName+".model", save_format="h5")

In [ ]:
optimizer = Adam()
train_model(model, optimizer, ds_train, ds_test, "Vgg")

Model summary :

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 